# DAY1(21/06/30)
## Tensorflow2 - keras API 모델 구현
## 학습 목적 : 이전 부트 캠프를 통해 기초 이론을 배웠던 NLP 분야를 공부하면서, 구현하는 방법이 여러가지가 있어서 이를 정리해보고자 진행함
### 1. Sequential Model
#### - Seqeuntial 모듈을 이용하여 간단한 순차적인 레이어의 스택을 구현할 수 있음. Seqeuntial 인스턴스를 생성하여 모듈을 선언함.
#### - 하지만, 모델의 층들이 순차적으로 구성돼 있지 않은 경우에는 Sequential 모델 구현에 제약이 있음.
#### - Sequential 모델은 하나의 플로우만 계산이 가능한데, 두개의 플로우를 계산한 후 합치는 경우엔 해당 모델을 쓸 수 없음.
### 2. Functional Model
#### - 모델의 스택 구조가 복잡할 경우에 쓰이는 방법.
#### - 다중 입력값 모델 / 다중 출력값 모델 / 공유 층을 활용하는 모델 / 데이터 흐름이 순차적이지 않은 모델일 경우에 사용함.
#### - 입력값을 받는 Input 모듈을 선언하여 모델을 시작함. 
### 3. Subclassing Model
#### - 가장 자유도가 높은 방법으로, tf.keras.Model을 상속받고 모델 내부 연산들을 직접 구현하면 됨.
#### - 이 방법은 Pytorch 프레임 워크에서 모델을 구현할 때 사용하는 방식과 유사함.
#### - 실제 Github에서 코드를 참조하면 이러한 방식이 많이 활용되는 것을 직접 확인하였음.


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

### 1. 데이터 샘플 생성 및 텍스트 전처리

In [ ]:
samples = ['너 오늘 이뻐 보인다',
           '나는 오늘 기분이 더러워',
           '끝내주는데, 좋은 일이 있나봐',
           '나 좋은 일이 생겼어',
           '아 오늘 진짜 짜증나',
           '환상적인데, 정말 좋은것 같아']

labels = [[1],[0],[1],[1],[0],[1]]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(samples)

In [ ]:
sequences = tokenizer.texts_to_sequences(samples)

In [ ]:
sequences

[[4, 1, 5, 6],
 [7, 1, 8, 9],
 [10, 2, 3, 11],
 [12, 2, 3, 13],
 [14, 1, 15, 16],
 [17, 18, 19, 20]]

In [ ]:
word_index = tokenizer.word_index
word_index

{'같아': 20,
 '기분이': 8,
 '끝내주는데': 10,
 '나': 12,
 '나는': 7,
 '너': 4,
 '더러워': 9,
 '보인다': 6,
 '생겼어': 13,
 '아': 14,
 '오늘': 1,
 '이뻐': 5,
 '일이': 3,
 '있나봐': 11,
 '정말': 18,
 '좋은': 2,
 '좋은것': 19,
 '진짜': 15,
 '짜증나': 16,
 '환상적인데': 17}

In [ ]:
batch = 2
num_epochs = 100
vocab_size = len(word_index)+1
emb_size = 128
hidden_dimension = 256
output_dimension = 1

### 2. Sequential 모델 생성 및 구현

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, emb_size, input_length = 5))
model.add(tf.keras.layers.Lambda(lambda x : tf.reduce_mean(x, axis = 1)))
model.add(tf.keras.layers.Dense(hidden_dimension, activation = 'relu'))
model.add(tf.keras.layers.Dense(output_dimension, activation = 'sigmoid'))

##tf.reduce_mean - 차원을 줄이면서 연산하는 함수로, 특정 차원을 제거하고 평균 값을 구함.
##위의 코드에서 tf.reduct_mean을 사용한 이유는 입력값을 임베딩 layer를 추가하고, 이후 각 단어의 벡터를 평균으로 구하기 위함.
##tf.reduct_sum - 차원을 줄이면서 연산하는 함수로, 특정 차원을 제거하고 합을 구함.

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
model.fit(sequences, labels, epochs=num_epochs, batch_size=1)

Epoch 1/100
6/6 [==============================] - 0s 2ms/step - loss: 0.6879 - accuracy: 0.6667
Epoch 2/100
6/6 [==============================] - 0s 2ms/step - loss: 0.6626 - accuracy: 1.0000
Epoch 3/100
6/6 [==============================] - 0s 3ms/step - loss: 0.6329 - accuracy: 1.0000
Epoch 4/100
6/6 [==============================] - 0s 2ms/step - loss: 0.6025 - accuracy: 1.0000
Epoch 5/100
6/6 [==============================] - 0s 2ms/step - loss: 0.5614 - accuracy: 1.0000
Epoch 6/100
6/6 [==============================] - 0s 2ms/step - loss: 0.5103 - accuracy: 1.0000
Epoch 7/100
6/6 [==============================] - 0s 3ms/step - loss: 0.4483 - accuracy: 1.0000
Epoch 8/100
6/6 [==============================] - 0s 2ms/step - loss: 0.3751 - accuracy: 1.0000
Epoch 9/100
6/6 [==============================] - 0s 3ms/step - loss: 0.2996 - accuracy: 1.0000
Epoch 10/100
6/6 [==============================] - 0s 2ms/step - loss: 0.2288 - accuracy: 1.0000
Epoch 11/100
6/6 [===========

### 3. Functial 모델 생성 및 구현

In [ ]:
inputs = tf.keras.layers.Input(shape = (4, ))
embed_output = tf.keras.layers.Embedding(vocab_size, emb_size)(inputs)
pooled_out = tf.reduce_mean(embed_output, axis = 1)
hidden_layer = tf.keras.layers.Dense(hidden_dimension, activation = 'relu')(pooled_out)
outputs = tf.keras.layers.Dense(output_dimension, activation = 'sigmoid')(hidden_layer)
model = tf.keras.Model(inputs = inputs, outputs = outputs)

model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model.fit(sequences, labels, epochs=num_epochs, batch_size=batch)

Epoch 1/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6978 - accuracy: 0.1667
Epoch 2/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6785 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6635 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6485 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6328 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6138 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 3ms/step - loss: 0.5917 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 3ms/step - loss: 0.5654 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5331 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 3ms/step - loss: 0.4970 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

### 4. Subclassing(Custom Model) 모델 생성 및 구현

In [ ]:
class CustomModel(tf.keras.Model):

    def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
        super(CustomModel, self).__init__(name = 'my_model')
        self.embedding = tf.keras.layers.Embedding(vocab_size, embed_dimension)
        self.dense_layer = tf.keras.layers.Dense(hidden_dimension, activation = 'relu')
        self.output_layer = tf.keras.layers.Dense(output_dimension, activation = 'sigmoid')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.reduce_mean(x, axis = 1)
        x = self.dense_layer(x)
        x = self.output_layer(x)

        return x

model = CustomModel(vocab_size, emb_size, hidden_dimension, output_dimension)

model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model.fit(sequences, labels, batch_size=batch, epochs=num_epochs)

Epoch 1/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6901 - accuracy: 0.6667
Epoch 2/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6707 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6528 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6344 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6140 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 6ms/step - loss: 0.5903 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 3ms/step - loss: 0.5634 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5320 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 4ms/step - loss: 0.4933 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 2ms/step - loss: 0.4525 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

### 5. 느낀 점
#### tensorflow2를 통해 모델을 구현하는 방법이 대표적으로 3가지가 있고, 어떻게 구현하는지 기본적인 방식을 정리할 수 있는 시간이었다.
#### 일주일 간, NLP를 배우면서 방대한 내용들이 머리 속에 들어왔다. 하지만, 스스로 정리가 안되고 새로운 지식들만 꾸겨넣고 있었다.
#### 지속적으로, 머릿속의 내용들을 정리하는 시간을 가져야겠다.
#### 그리고, Subclassing 방법이 Github에서 많이 활용되고 있는 것을 봤기 때문에, 이 방법이 효율적일 수 있겠다라는 생각이 들고 많이 생소한 문법이기 때문에 이에 적응하도록 노력해야겠다.